#### CSCE 670 :: Information Storage and Retrieval :: Texas A&M University :: Spring 2018


# Homework 3:  Embeddings + Recommenders

### 100 points [5% of your final grade]

### Due: Monday, April 9 by 11:59pm

*Goals of this homework:* There are two main learning objectives: (i) implement and evaluate a pre-cursor to modern word2vec embeddings; and (ii) implement, evaluate, and improve upon traditional collaborative filtering recommenders.

*Submission Instructions:* To submit your homework, rename this notebook as UIN_hw#.ipynb. For example, this homework submission would be: YourUIN_hw3.ipynb. Submit this notebook via ecampus. Your notebook should be completely self-contained, with the results visible in the notebook. 

*Late submission policy:* For this homework, you may use up to three of your late days, meaning that no submissions will be accepted after Thursday, April 12 at 11:59pm.

In [24]:
# All the imported libraries
import numpy as np
from scipy.stats import pearsonr
from sklearn.cluster import KMeans

# Part 1: Word Embeddings (50 points)
For this first part, we're going to implement a word embedding approach that is a bit simpler than word2vec. The key idea is to look at co-occurrences between center words and context words (somewhat like in word2vec) but without any pesky learning of model parameters.

If you're interested in a deeper treatment of comparing count vs. learned embeddings, take a look at: [Don’t count, predict! A systematic comparison of
context-counting vs. context-predicting semantic vectors](
http://www.aclweb.org/anthology/P14-1023)

## Load the Brown Corpus

The dataset for this part is the (in)famous [Brown corpus](https://en.wikipedia.org/wiki/Brown_Corpus) that is a collection of text samples from a wide range of sources, with over one million unique words. Good for us, you can find the Brown corpus in nltk. *Make sure you have already installed nltk with something like: conda install nltk*

In [25]:
import nltk
nltk.download('brown')

[nltk_data] Downloading package brown to /home/abhilash/nltk_data...
[nltk_data]   Package brown is already up-to-date!


True

Once you have it locally, you can load the dataset into your notebook. You can access the words using brown.words():

In [26]:
from nltk.corpus import brown
brown.words()

[u'The', u'Fulton', u'County', u'Grand', u'Jury', ...]

## 1.1 Dataset Pre-processing
OK, now we need to do some basic pre-processing. For this part you should:

* Remove stopwords and punctuation.
* Make everything lowercase.

Then, count how often each word occurs. We will define the 5,000 most  frequent words as your vocabulary (V). We will define the 1,000 most frequent words as our context (C). Include a print statement below to show the top-20 words after pre-processing.

In [27]:
from nltk.corpus import brown
import string

# Remove stop words
from nltk.corpus import stopwords
stop_words = stopwords.words(fileids='english')
sentence = 'The quick brown fox was too quick for the white fox brown quick \
fox'
listWords = sentence.split()
listWords = brown.words()
brown_noStopWords = [w for w in listWords if w.lower() not in stop_words]

# Remove punctuation
import re
brown_noPunctuation = [re.sub(r'[^\w]','',w) for w in brown_noStopWords]
# Remove the leftover empty strings, so that they don't get counted in the
# window operation
brown_noPunctuation = [w for w in brown_noPunctuation if w != ''] 

# stem the words
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
brown_preprocessed = [ stemmer.stem(w) for w in brown_noPunctuation ]

In [28]:
# The 5000 most common words are the vocabulary and the 1000 most common words
# are the Context.
dictWordCnt = {}
for token in brown_preprocessed:
    if token in dictWordCnt:
        dictWordCnt[token] += 1
    else:
        dictWordCnt[token] = 1
# sort the dictionary and return the keys in decreasing order of word frequency
cnt = 0
listSortedKeys = sorted(dictWordCnt, key=dictWordCnt.get, reverse=True)
for w in listSortedKeys:
    cnt += 1
    print (w + ': {}'.format(dictWordCnt[w]))
    if cnt >= 20:
        break

one: 3480
would: 2714
said: 1961
time: 1930
year: 1659
new: 1647
could: 1601
state: 1562
like: 1535
use: 1476
may: 1419
two: 1414
first: 1361
man: 1352
even: 1319
make: 1225
work: 1178
made: 1125
day: 1088
also: 1069


In [29]:
dictVocab2idx = {}
dictContext2idx = {}
dictIdx2Vocab = {}
dictIdx2Context = {}
for idx,token in enumerate(listSortedKeys[:5000]):
    dictVocab2idx[token] = idx
    dictIdx2Vocab[idx] = token
    if idx < 1000:
        dictIdx2Context[idx] = token
        dictContext2idx[token] = idx

## 1.2 Building the Co-occurrence Matrix 

For each word in the vocabulary (w), we want to calculate how often context words from C appear in its surrounding window of size 4 (two words before and two words after).

In other words, we need to define a co-occurrence matrix that has a dimension of |V|x|C| such that each cell (w,c) represents the number of times c occurs in a window around w. 

In [30]:
VOCAB_SIZE = 5000
CONTEXT_SIZE = 1000
cooccurence_matrix = np.zeros(shape=(VOCAB_SIZE,CONTEXT_SIZE))
for index,w in enumerate(brown_preprocessed[2:-2]):
    actIndex = index + 2    # Actual index is 2 more than index
    # Check if the word is in Vocab, then check the two words prior to it and
    # the 2 words after it to see if they belong to the Context set. Increment
    # the (w,c) entry if they do.
    if w in dictVocab2idx:
        window_words = (brown_preprocessed[actIndex-2],brown_preprocessed[actIndex-1],
         brown_preprocessed[actIndex+1],brown_preprocessed[actIndex+2])
        for c in window_words:
            if c in dictContext2idx:
                cooccurence_matrix[dictVocab2idx[w]][dictContext2idx[c]] += 1
#print(cooccurence_matrix)

## 1.3 Probability Distribution

Using the co-occurrence matrix, we can compute the probability distribution Pr(c|w) of context word c around w as well as the overall probability distribution of each context word c with Pr(c).  

In [31]:
cwPDF = np.zeros(shape=(VOCAB_SIZE,CONTEXT_SIZE))
for wIdx,cwArray in enumerate(cooccurence_matrix):
    sumCw = sum(cwArray)
    for cIdx,val in enumerate(cwArray):
        cwPDF[wIdx][cIdx] = val/sumCw
# print(cwPDF)
sumC = np.sum(np.transpose(cooccurence_matrix),axis=1)
cPDF = sumC/np.sum(cooccurence_matrix)
# print(cPDF)

## 1.4 Embedding Representation

Now you can represent each vocabulary word as a |C| dimensional vector using this equation:

Vector(w)= max(0, log (Pr(c|w)/Pr(c)))

This is a traditional approach called *pointwise mutual information* that pre-dates word2vec by some time. 

In [33]:
cwPDF += 0.0000001
embedding = np.log(cwPDF/cPDF)
embedding = np.maximum(0,embedding)
# print(embedding)

## 1.5 Analysis

So now we have some embeddings for each word. But are they meaningful? For this part, you should:

- First, cluster the vocabulary into 100 clusters using k-means. Look over the words in each cluster, can you see any relation beween words? Discuss your observations.

- Second, for the top-20 most frequent words, find the nearest neighbors using cosine distance (1- cosine similarity). Do the findings make sense? Discuss.

In [34]:
kmeans = KMeans(n_clusters=100, random_state=0).fit(embedding)

In [81]:
clusterElemIndices = np.where(kmeans.labels_==20)[0]
listClusterElems = [dictIdx2Vocab[index] for index in clusterElemIndices]
print(listClusterElems)

[u'temperatur', u'fig', u'anod', u'16', u'ratio', u'fiber', u'arc', u'diamet', u'lb', u'cm', u'ft', u'mm']


The above words belong to the 20th cluster and it is clear that they represent measurements, the units like lb, cm, ft, mm and also the quantities like temperature, arc, diameter and ratio appear here. This makes sense because usually the quantities and their units of measurements occur together and are closely related.

The above method represents each word by the words around it, so words which have similar contexts will end having similar representations and get grouped in the same cluster.

There are also words like fig, anode and fiber in the same cluster which suggests that method is not perfect.

In [40]:
def compute_cosineSimilarity(a,b):
    return np.dot(a,b)/ (np.linalg.norm(a) * np.linalg.norm(b))

def compute_cosineDistance(u,v):
    return 1 - compute_cosineSimilarity(u,v)

print(listSortedKeys[:20])
for word in listSortedKeys[:20]:
    index = dictVocab2idx[word]
    wordEmbedding = embedding[index]
    listNearestNeighbors = []
    for i,e in enumerate(embedding):
        if compute_cosineDistance(wordEmbedding,e) < 0.47:
            listNearestNeighbors.append(dictIdx2Vocab[i])
    if len(listNearestNeighbors) > 1:
        print(listNearestNeighbors)

[u'one', u'would', u'said', u'time', u'year', u'new', u'could', u'state', u'like', u'use', u'may', u'two', u'first', u'man', u'even', u'make', u'work', u'made', u'day', u'also']
[u'said', u'know', u'ask', u'tell', u'talk', u'Im']
[u'year', u'month']
[u'two', u'three']
[u'day', u'week']


Nearest neighbors which make perfect sense :
- would, could, may, also : these usually appear together
- year, month
- two, three
- day, week

For the most part, it appears that the similar words are closer together in the embedding space which means that the word meanings are captured reasonably well.

However, things are not perfect though, for eg, seemingly unrelated words like first, man, day appear to be very close together. Perhaps, a better distance metric can provide a clearer picture of the embedding space.

# Part 2. Collaborative Filtering (50 points)

In this second part, you will implement collaborative filtering on the Netflix prize dataset -- don’t freak out, the provided sample dataset has only ~2000 items and ~28,000 users.

As background, read the paper [Empirical Analysis of Predictive Algorithms for Collaborative Filtering](https://arxiv.org/pdf/1301.7363.pdf) up to Section 2.1. Of course you can read further if you are interested, and you can also refer to the course slides for collaborative filtering.

## 2.1 Load Netflix Data

The dataset is subset of movie ratings data from the Netflix Prize Challenge. Download the dataset from Piazza. It contains a train set, test set, movie file, and README file. The last two files are original ones from the Netflix Prize, however; in this homework you will deal with train and test files which both are subsets of the Netflix training data. Each of train and test files has lines having this format: MovieID,UserID,Rating.

Your job is to predict a rating in the test set using those provided in the training set.

In [104]:
# load the data, then print out the number of ratings, 
# movies and users in each of train and test sets.
def create_utilityMatrix(ratings_file):
    """
    Computes the total number of unique users and items and returns the values
    """
    #The lines are in the format : movieId,userId,rating
    dictMovies2Index = {}
    dictUsers2Index = {}
    numRatings = 0
    movieIndex = 0
    userIndex = 0
    with open(ratings_file) as file:
        for line in file:
            movieId,userId,rating = line.split(',')
            numRatings += 1
            if userId not in dictUsers2Index:
                dictUsers2Index[userId] = userIndex
                userIndex += 1
            if movieId not in dictMovies2Index:
                dictMovies2Index[movieId] = movieIndex
                movieIndex += 1
    numUsers = userIndex
    numMovies = movieIndex
    print('Summary of {}'.format(ratings_file))
    print('Number of ratings : {}'.format(numRatings))
    print('Number of users : {}'.format(numUsers))
    print('Number of movies : {}\n'.format(numMovies))

    utility_matrix = np.zeros(shape=(numUsers,numMovies))
    with open(ratings_file) as file:
        for line in file:
            movieId,userId,rating = line.split(',')
            user = dictUsers2Index[userId]
            movie = dictMovies2Index[movieId]
            utility_matrix[user][movie] = rating
    return utility_matrix, dictUsers2Index, dictMovies2Index

training_file = 'netflix-dataset/TrainingRatings.txt'
testing_file = 'netflix-dataset/TestingRatings.txt'
utility_matrix, dictUsers2Index, dictMovies2Index = create_utilityMatrix(training_file)
_ = create_utilityMatrix(testing_file)


Summary of netflix-dataset/TrainingRatings.txt
Number of ratings : 3255352
Number of users : 28978
Number of movies : 1821

Summary of netflix-dataset/TestingRatings.txt
Number of ratings : 100478
Number of users : 27555
Number of movies : 1701



## 2.2 Implement CF

In this part, you will implement the basic collaborative filtering algorithm described in Section 2.1 of the paper -- that is, focus only on Equations 1 and 2 (where Equation 2 is just the Pearson correlation). You should consider the first 5,000 users with their associated items in the test set. 

Note that you should test the algorithm for a small set of users e.g., 10 users first and then run for 5,000 users. It may take long to run but you won't have memory issues. 

Set k to 0.1. 

Finding the right value for k proved to be to tedious and I simply could not find a suitable value. So, instead I used the following equation :

**p<sub>a,j</sub> = avg(v<sub>a</sub>) + avg(v<sub>i,j</sub> - mean(v<sub>i</sub>))**

**v<sub>i,j</sub> - mean(v<sub>i</sub>)** represents the net preference of user i for item j w.r.t her mean rating value.

The idea is to correct the user average by collective preference of the similar users.

In [45]:
def evaluate(utility_matrix, dictUsers2Index, dictMovies2Index, testing_file):
    """
    For every user,movie pair in Testing set, find the set of the most similar users in
    the training set who have rated movie and compute the average rating
    """
    def get_commonRatings(ratings, mask):
        common_ratings = ratings * mask
        # Remove the zero elements, these lead to a higher correlation value
        # than the reality
        return common_ratings[common_ratings != 0]

    predictions = []
    expected_values = []
    import sys
    sys.stdout.flush()
    with open(testing_file) as file:
        for line in file:
            stripped_line = line.strip() # Remove leading/trailing whitespace
            movieId, userId, rating = stripped_line.split(',')
            expected_values.append(float(rating))
            print("predicting {}'s rating for {}".format(userId,movieId))
            activeUserIndex = dictUsers2Index[userId]
            activeMovieIndex = dictMovies2Index[movieId]
            # Create a mask for the active user, indicating the rated items
            ratings_activeUser = utility_matrix[activeUserIndex]
            mask = ratings_activeUser != 0
            numSimilarUsers = 0
            sum_similarUserRatings = 0
            # Initialize the predicted rating with the average rating value for
            # the active user, later this is adjusted with the collaborative
            # value
            pred_rating = ratings_activeUser[ratings_activeUser != 0].mean()
            for ratings_user in utility_matrix:
                if ratings_user[activeMovieIndex] != 0:
                    # To measure the similarity, we need to only compare the
                    # items rated by both users, so create a mask of common
                    # items
                    currUser_mask = mask * (ratings_user != 0)
                    # if the mask contains only 2 items, pearson correlation
                    # always returns 1 which is not very useful
                    if (sum(currUser_mask) > 2):
                        # Get the ratings of the common items
                        user_commonRatings = get_commonRatings(
                            ratings_user, currUser_mask)
                        activeUser_commonRatings = get_commonRatings(
                            ratings_activeUser, currUser_mask)
                        pearson_correlation = pearsonr(activeUser_commonRatings, user_commonRatings)[0]
                      #  print(user_commonRatings, activeUser_commonRatings,
                      #        pearson_correlation)
                        if pearson_correlation > 0.7:
                            print("active user and similar user's rating : {} and {}".format(activeUser_commonRatings, user_commonRatings))
                            numSimilarUsers += 1
                            sum_similarUserRatings += ( ratings_user[activeMovieIndex] - user_commonRatings.mean())
            pred_rating += (sum_similarUserRatings / (numSimilarUsers + 0.0001))
            predictions.append(pred_rating)
        predictions = np.array(predictions)
        expected_values = np.array(expected_values)
        print(predictions, expected_values)
        return predictions, expected_values

## 2.3 Evaluation 

You should evaluate your predictions using Mean Absolute Error and Root Mean Squared Error. 

In [42]:
training_file = 'netflix-dataset/TrainingRatings.txt'
testing_file = 'netflix-dataset/TestingRatings_small.txt'

In [43]:
def compute_mae(predictions, expected_values):
    return (np.absolute(predictions - expected_values)).mean()

In [44]:
def compute_rmse(predictions, expected_values):
    return np.sqrt(((predictions - expected_values) ** 2).mean())

I ran the model for the first 600 users in the Testing Set and got the following results:  
**rmse : 1.092**  
**mae  : 0.852**

## 2.4 Extensions

Given your results in the previous part, can you do better? For this last part you should report on your best attempt at improving MAE and RMSE. Provide code, results, plus a brief discussion on your approach.

I tried using cosine similarity with mean subtraction instead of pearson correlation

In [46]:
def compute_cosineSimilarity(a,b):
    return np.dot(a,b)/ (np.linalg.norm(a) * np.linalg.norm(b))

def compute_similarity(u, v):
    u_m = u[u != 0].mean()
    v_m = v[v != 0].mean()
    u_mask = (u != 0) * u_m
    v_mask = (v != 0) * v_m
    return compute_cosineSimilarity(u - u_mask, v - v_mask)

I got the following results :  
**rmse : 1.092**  
**mae  : 0.852**  
This is exactly the same as before which, in retrospect, is unsurprising since cosine similarity with mean subtraction is the same as pearson correlation!  
Nonetheless, this was my best attempt at improving RMSE and all other attempts ended up worse.